In [17]:
from typedspark import (
    Column,
    Schema,
    create_partially_filled_dataset,
    register_schema_to_dataset,
    transform_to_schema,
)

from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, StringType

spark = SparkSession.Builder().getOrCreate()


class Person(Schema):
    id: Column[IntegerType]
    name: Column[StringType]
    age: Column[IntegerType]


class Job(Schema):
    id: Column[IntegerType]
    salary: Column[IntegerType]


df_a = create_partially_filled_dataset(spark, Person, {Person.id: [1, 2, 3]})
df_b = create_partially_filled_dataset(spark, Job, {Job.id: [1, 2, 3]})

person = register_schema_to_dataset(df_a, Person)
job = register_schema_to_dataset(df_b, Job)


class PersonWithJob(Person, Job):
    pass


(
    transform_to_schema(
        df_a.join(
            df_b,
            person.id == job.id,
        ),
        PersonWithJob,
        {
            PersonWithJob.id: job.id,
        },
    ).show()
)

+---+------+----+----+
| id|salary|name| age|
+---+------+----+----+
|  1|  null|null|null|
|  2|  null|null|null|
|  3|  null|null|null|
+---+------+----+----+



23/08/27 18:23:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 195895 ms exceeds timeout 120000 ms
23/08/27 18:23:47 WARN SparkContext: Killing executors is not supported by current scheduler.
23/08/27 18:23:52 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B